In [13]:
import numpy as np 
import pandas as pd
#import calendar
import math

# plots
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

In [14]:
filenames = ['properties/properati-AR-2016-03-01-properties-sell-six_months.csv',
            'properties/properati-AR-2016-04-01-properties-sell.csv',
            'properties/properati-AR-2016-05-01-properties-sell.csv',
            'properties/properati-AR-2016-06-01-properties-sell.csv',
            'properties/properati-AR-2016-07-01-properties-sell.csv',
            'properties/properati-AR-2016-08-01-properties-sell.csv',
            'properties/properati-AR-2016-09-01-properties-sell.csv',
            'properties/properati-AR-2016-10-01-properties-sell.csv',
            'properties/properati-AR-2016-11-01-properties-sell.csv',
            'properties/properati-AR-2016-12-01-properties-sell.csv', 
            'properties/properati-AR-2017-01-01-properties-sell.csv',
            'properties/properati-AR-2017-02-01-properties-sell.csv',
            'properties/properati-AR-2017-08-01-properties-sell-six_months.csv']

dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))
    
#Uno los dataFrame
metadata = pd.DataFrame()
for dataframe in dataframes:
    metadata = pd.concat([metadata, dataframe]).drop_duplicates().reset_index(drop=True)

In [16]:
data = metadata.loc[(metadata.state_name.str.contains('Buenos Aires')) | (metadata.state_name.str.contains\
                ('Capital Federal')),:]
data.shape
# 1.283.741 entries antes, ahora 115928

(115928, 27)

In [17]:
test = pd.read_csv('properties/properati_dataset_testing_noprice.csv')
test_visualiz = test.loc[:,['state_name', 'id']].groupby('state_name').agg(np.size)
test_visualiz.head(10)

,id
state_name,
Bs.As. G.B.A. Zona Norte,2
Bs.As. G.B.A. Zona Oeste,3101
Bs.As. G.B.A. Zona Sur,2215
Buenos Aires Interior,1
Capital Federal,8847


In [18]:
data['state_name'].tail()

296466                 Capital Federal
296472                 Capital Federal
296474                 Capital Federal
296475    Buenos Aires Costa Atlántica
296476                 Capital Federal
Name: state_name, dtype: object

In [19]:
# Depuración del set de test
#Completo los valores NaN con promedios para realizar la predicción
prom_surface_total_in_m2 = test['surface_total_in_m2'].mean()
prom_surface_covered_in_m2 = test['surface_covered_in_m2'].mean()
prom_floor = int(test['floor'].mean())
prom_rooms = int(test['rooms'].mean())
prom_lat = test['lat'].mean()
prom_lon = test['lon'].mean()

test_float = test
test_float['surface_total_in_m2'].fillna(prom_surface_total_in_m2, inplace=True)
test_float['surface_covered_in_m2'].fillna(prom_surface_covered_in_m2, inplace=True)
test_float['floor'].fillna(prom_floor, inplace=True)
test_float['rooms'].fillna(prom_rooms, inplace=True)
test_float['lat'].fillna(prom_lat, inplace=True)
test_float['lon'].fillna(prom_lon, inplace=True)


test_float.head(100)

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.000000,359.471588,10.0,2.0,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,-34.629923,-58.465820,0.000000,359.471588,10.0,2.0,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.000000,48.000000,10.0,2.0,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,10.0,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,51.000000,51.000000,10.0,2.0,NaN,LIVING COMEDOR CON PISOS DE PO...
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,10.0,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,10.0,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,39.000000,39.000000,10.0,2.0,NaN,AMBIENTE DIVISIBLE CON PISOS D...
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.000000,67.000000,10.0,2.0,2600,EXCELENTE 2 AMB- APTO PROFESIO...
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,-34.629923,-58.465820,47.000000,47.000000,10.0,2.0,NaN,IMPECABLE MONOAMBIENTE- DIVISI...


In [21]:
test_float.count()

id                         14166
created_on                 14166
property_type              14166
operation                  14166
place_name                 14166
place_with_parent_names    14166
country_name               14166
state_name                 14166
lat-lon                    10487
lat                        14166
lon                        14166
surface_total_in_m2        14166
surface_covered_in_m2      14166
floor                      14166
rooms                      14166
expenses                    2543
description                14166
dtype: int64

<br><B><FONT FACE="Arial" SIZE="5">Prediccion usando XGBoosting'</FONT></B><br/>

In [20]:
#FILTRO LOS DATOS ANOMALOS

data_float = data.loc[:, ['price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',\
                          'floor', 'rooms', 'lat', 'lon']].dropna(axis= 0, how='any')

#filtro
data_float = \
data_float.loc[((data_float.surface_total_in_m2 >= 0.0) & (data_float.surface_total_in_m2 <= 1000000.0)) & \
               ((data_float.surface_covered_in_m2 >= 0.0) & (data_float.surface_covered_in_m2 <= 50000000.0)) & \
               ((data_float.floor >= 0.0) & (data_float.floor <= 200.0)) & \
               ((data_float.rooms >= 0.0) & (data_float.rooms <= 200.0)), :]


data_float.shape
#(3772, 7)

(3772, 7)

In [22]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [23]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls', 'criterion': 'mse'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [24]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.838/0.717


Precisión Gradient Boosting train/test  0.838/0.717


In [25]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms']])
test['price_aprox_usd_XGB'].head(100)

0     926106.125274
1     926106.125274
2     117013.529525
3     101878.130135
4     121822.631846
5     101878.130135
6     101878.130135
7     101878.130135
8     177497.746930
9     107821.212080
10    107821.212080
11    107821.212080
12    177497.746930
13    398719.359538
14    101878.130135
15    101878.130135
16    101878.130135
17    101878.130135
18    101878.130135
19    101878.130135
20    101878.130135
21    101878.130135
22    116224.539298
23    121822.631846
24    121822.631846
25    121822.631846
26    121822.631846
27    101878.130135
28    101878.130135
29    101878.130135
          ...      
70    140923.489501
71    130469.405011
72    292375.380703
73    292375.380703
74    140923.489501
75    140923.489501
76    140923.489501
77    140923.489501
78    241158.456116
79    140923.489501
80    250418.157121
81    241158.456116
82    302420.150820
83    215359.178513
84    215676.264068
85    115577.187629
86    275756.905626
87    108515.179103
88    236972.790448


In [26]:
#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingBuenosAires.csv', index=False)

<br><B><FONT FACE="Arial" SIZE="5">XGBoosting con lat y lon</FONT></B><br/>

In [153]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms', 'lat', 'lon']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [154]:
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

params2 = {'random_state': 0, 'n_estimators': 500,
                                  'max_depth': 1, 'learning_rate': 0.01}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=500,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [155]:
#Me fijo la precision
#random_state=0, n_estimators=500, max_depth=1, learning_rate=0.01
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.905/0.736


Precisión Gradient Boosting train/test  0.905/0.736


In [156]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
#test_float_reduced.to_csv('SubmitXGBoostingLatLonGenerico.csv', index=False)
test_float_reduced['price_usd'].head(100)

0     1.324732e+06
1     8.981894e+05
2     1.413863e+05
3     9.050579e+04
4     1.175260e+05
5     9.050579e+04
6     9.050579e+04
7     9.050579e+04
8     1.764229e+05
9     9.167031e+04
10    9.167031e+04
11    9.167031e+04
12    1.333397e+05
13    3.978144e+05
14    9.050579e+04
15    9.050579e+04
16    9.050579e+04
17    9.050579e+04
18    9.050579e+04
19    9.050579e+04
20    9.050579e+04
21    9.050579e+04
22    1.099744e+05
23    1.175260e+05
24    1.175260e+05
25    1.175260e+05
26    1.175260e+05
27    9.050579e+04
28    9.050579e+04
29    1.018247e+05
          ...     
70    1.260256e+05
71    1.536789e+05
72    2.086361e+05
73    2.086361e+05
74    1.260256e+05
75    1.260256e+05
76    1.260256e+05
77    1.260256e+05
78    3.030248e+05
79    1.260256e+05
80    1.854622e+05
81    3.030248e+05
82    2.072858e+05
83    1.628461e+05
84    2.160333e+05
85    9.778646e+04
86    2.596888e+05
87    1.161159e+05
88    1.809908e+05
89    3.158124e+05
90    1.619645e+05
91    3.2618

<br><B><FONT FACE="Arial" SIZE="5">Uso grid-search para estimar los parametros</FONT></B><br/>

In [157]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',  'lat', 'lon']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [158]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

param_test1 = {'n_estimators':list(range(1,600,10))}

gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=500,\
min_samples_leaf=50, max_depth=8, max_features='sqrt', subsample=0.8, random_state=10),\
param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)

#gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1 ,max_features='sqrt',\
#subsample=0.8, random_state=10), param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
                        
gsearch1.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=8,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=50,
             min_samples_split=500, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=10,
             subsample=0.8, verbose=0, warm_start=False),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [1, 11, 21, 31, 41, 51, 61, 71, 81, 91, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 201, 211, 221, 231, 241, 251, 261, 271, 281, 291, 301, 311, 321, 331, 341, 351, 361, 371, 381, 391, 401, 411, 421, 431, 441, 451, 461, 471, 481, 491, 501, 511, 521, 531, 541, 551, 561, 571, 581, 591]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring='neg_mean_squared_error', verbose=0)

In [159]:
#Me fijo el resultado del parametro
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: -16843996650.16515, std: 5166367723.16848, params: {'n_estimators': 1},
  mean: -10106486809.18195, std: 4082827112.22908, params: {'n_estimators': 11},
  mean: -8538780870.45991, std: 3526678707.50090, params: {'n_estimators': 21},
  mean: -7936005575.74576, std: 3366790529.86906, params: {'n_estimators': 31},
  mean: -7721878509.57545, std: 3384967348.68345, params: {'n_estimators': 41},
  mean: -7578898578.72201, std: 3399084462.84676, params: {'n_estimators': 51},
  mean: -7434293526.88597, std: 3384514884.52790, params: {'n_estimators': 61},
  mean: -7327403918.33173, std: 3395215442.16270, params: {'n_estimators': 71},
  mean: -7143791592.42634, std: 3395911735.19798, params: {'n_estimators': 81},
  mean: -7084008818.14432, std: 3360934724.76022, params: {'n_estimators': 91},
  mean: -7054718895.18933, std: 3424051808.30917, params: {'n_estimators': 101},
  mean: -6926375854.88223, std: 3365949829.00404, params: {'n_estimators': 111},
  mean: -6856580232.21160, std: 34190

In [160]:
#Encontrar los parametros max_depth y min_samples_split
#Aca ya uso n_estimators=581
param_test2 = {'max_depth':list(range(2,16,2)), 'min_samples_split':list(range(200,1001,10))}
gsearch2 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=581, max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test2, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -7375194367.04868, std: 4754377379.50517, params: {'min_samples_split': 200, 'max_depth': 2},
  mean: -7295154340.62962, std: 4716818993.67892, params: {'min_samples_split': 210, 'max_depth': 2},
  mean: -7473914787.29518, std: 4987473907.39484, params: {'min_samples_split': 220, 'max_depth': 2},
  mean: -7316928955.07467, std: 4757618791.19247, params: {'min_samples_split': 230, 'max_depth': 2},
  mean: -7458095523.75313, std: 4595570798.56130, params: {'min_samples_split': 240, 'max_depth': 2},
  mean: -7457553111.44466, std: 4641693597.84215, params: {'min_samples_split': 250, 'max_depth': 2},
  mean: -7477191439.44407, std: 4640431357.06419, params: {'min_samples_split': 260, 'max_depth': 2},
  mean: -7448129403.08715, std: 4569122833.77777, params: {'min_samples_split': 270, 'max_depth': 2},
  mean: -7727959919.22849, std: 5006604957.95669, params: {'min_samples_split': 280, 'max_depth': 2},
  mean: -7688629166.66718, std: 4813212695.97451, params: {'min_samples_split': 29

In [161]:
#Encontrar el parametro min_samples_leaf
# 'n_estimators':581, 'max_depth': 8, 'min_samples_split': 270

param_test3 = {'min_samples_leaf':list(range(10,100,1))}
gsearch3 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=581, min_samples_split=270, max_depth=8,max_features='sqrt', subsample=0.8, random_state=10), 
param_grid = param_test3, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train, y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

([mean: -8399216030.94999, std: 6599000044.92985, params: {'min_samples_leaf': 10},
  mean: -8597673168.53061, std: 6425154526.77927, params: {'min_samples_leaf': 11},
  mean: -8107424841.76738, std: 5697225640.34723, params: {'min_samples_leaf': 12},
  mean: -7874081723.43711, std: 5747400782.89956, params: {'min_samples_leaf': 13},
  mean: -7722841007.55219, std: 5409935736.66887, params: {'min_samples_leaf': 14},
  mean: -7489178426.01521, std: 5008901450.31734, params: {'min_samples_leaf': 15},
  mean: -7317507718.49198, std: 4733091010.32380, params: {'min_samples_leaf': 16},
  mean: -6949367523.22112, std: 4551433073.68125, params: {'min_samples_leaf': 17},
  mean: -6917916370.21925, std: 4455527856.72187, params: {'min_samples_leaf': 18},
  mean: -6827995560.03359, std: 4431037944.11937, params: {'min_samples_leaf': 19},
  mean: -6743158985.26892, std: 4362267314.50375, params: {'min_samples_leaf': 20},
  mean: -6739368845.78295, std: 4402597301.48444, params: {'min_samples_leaf

In [162]:
#Encontrar el parametro max_features
# 'n_estimators':581, 'max_depth': 8, 'min_samples_split': 270, 'min_samples_leaf': 56

param_test4 = {'max_features':list(range(1,4,1))}
gsearch4 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, n_estimators=581, max_depth=8, min_samples_split=270, min_samples_leaf=56, subsample=0.8, random_state=10),
param_grid = param_test4, scoring='neg_mean_squared_error',n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train, y_train)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

([mean: -6066090695.22114, std: 3630024225.79246, params: {'max_features': 1},
  mean: -5938948164.06238, std: 3642771561.16862, params: {'max_features': 2},
  mean: -6059849674.49138, std: 3652271778.65611, params: {'max_features': 3}],
 {'max_features': 2},
 -5938948164.062378)

In [164]:
# 'n_estimators':204, 'max_depth': 14, 'min_samples_split': 300, 'min_samples_leaf': 44, 'max_features': 2
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 581, 'max_depth': 8, 'min_samples_split': 270, 'min_samples_leaf': 56, 'max_features': 2,
          'learning_rate': 0.1}


clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

#Me fijo la precision
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.823/0.767

Precisión Gradient Boosting train/test  0.884/0.756


In [165]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS.csv', index=False)

## Uso grid-search para estimar los parametros, ésta vez todos juntos


In [ ]:
#Creo X e Y
feature_cols = ['surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',  'lat', 'lon']
TRAIN = data_float[:2600]
TEST = data_float[2600:]

#Separo en set de entrenamiento y set de Test
X_train = TRAIN[feature_cols]
X_test = TEST[feature_cols]

y_train = TRAIN.price_aprox_usd
y_test = TEST.price_aprox_usd

In [ ]:
#Grid-search para encontrar el parametro n_estimators
#Arranco con parametros normales al azar.
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

param_test1 = {'n_estimators':list(range(1,500,5)), 'max_depth':list(range(2,16,2)),\
               'min_samples_split':list(range(200,1000,50)), 'min_samples_leaf':list(range(10,100,5)),\
                'max_features':list(range(1,4,1))}

gsearch1 = GridSearchCV(estimator = GradientBoostingRegressor(learning_rate=0.1, subsample=0.8, random_state=10),\
                param_grid = param_test1, scoring='neg_mean_squared_error', n_jobs=4,iid=False, cv=5)
                        
gsearch1.fit(X_train, y_train)

In [ ]:
# {'max_depth': 10,
#  'max_features': 3,
#  'min_samples_leaf': 40,
#  'min_samples_split': 200,
#  'n_estimators': 261},

gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

### Con estos hiper-parametros estimados, hago una prediccion

In [ ]:
# 'n_estimators':261, 'max_depth': 10, 'min_samples_split': 200, 'min_samples_leaf': 40, 'max_features': 3
#Voy a realizar una prueba con estos parametros
#Entreno el modelo
from sklearn import ensemble
params = {'n_estimators': 261, 'max_depth': 10, 'min_samples_split': 200, 'min_samples_leaf': 40, 'max_features': 3,
          'learning_rate': 0.1}


clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)

#Me fijo la precision
print('Precisión Gradient Boosting train/test  {0:.3f}/{1:.3f}'
      .format(clf.score(X_train, y_train), clf.score(X_test, y_test)))

#Precisión Gradient Boosting train/test  0.857/0.837

In [ ]:
#Hago la prediccion para Kaggle
test['price_aprox_usd_XGB'] = clf.predict(test.loc[:, ['surface_total_in_m2', 'surface_covered_in_m2',\
                                        'floor', 'rooms',  'lat', 'lon']])

#Genero el csv con los nombres de columnas requeridos por Kaggle
test_float_reduced = test_float.loc[:,['id', 'price_aprox_usd_XGB']]
test_float_reduced.columns=['id', 'price_usd']
test_float_reduced.to_csv('SubmitXGBoostingGS2.csv', index=False)